In [2]:
import pandas as pd
import numpy as np

import psycopg2
import sqlalchemy

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set(font_scale=1.5)

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# DSN (data source name) format for database connections:  
# [protocol / database  name]://[username]:[password]@[hostname / ip]:[port]/[database name here]

# on your computer you are the user postgres (full administrative access)
db_user = 'postgres'
# if you need a password to access a database, put it here
db_password = ''
# on your computer, use localhost
db_host = 'localhost'
# the default port for postgres is 5432
db_port = 5432
# we want to creat a database with the following name
database = 'final_db'
#set up a connection to postgres without specifying the database
# conn_postgres = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}'

conn_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{database}'
conn = psycopg2.connect(conn_str)

# connect using sqlalchemy
engine = sqlalchemy.engine.create_engine(conn_str)

# connect using sqlalchemy
connection = engine.connect()

# # commit
# connection.execute('commit')

# close connection
connection.close()

# open a new connection to the database that we created
database = 'final_db'
conn_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{database}'
conn = psycopg2.connect(conn_str)

# connect using sqlalchemy
engine = sqlalchemy.engine.create_engine(conn_str)

In [2]:
# Each row represents a player's performance & details about the game

query = """

SELECT c.nba_game_date, c.nba_season_id, e.br_location, c.nba_gm_type, e.br_outcome, c.nba_plus_minus home_plus_minus,  a.*, d.birthdate, d.nba_height, b.nba_e_off_rating, b.nba_off_rating, b.nba_e_def_rating, b.nba_def_rating,
b.nba_e_net_rating, b.nba_net_rating, b.nba_ast_pct, b.nba_ast_tov, b.nba_ast_ratio,
b.nba_oreb_pct, b.nba_dreb_pct, b.nba_reb_pct, b.nba_tm_tov_pct, b.nba_efg_pct,
b.nba_ts_pct, b.nba_usg_pct, b.nba_e_pace, b.nba_pace, b.nba_pace_per40, b.nba_poss, b.nba_pie, e.br_game_score
FROM nba_tra_box a
LEFT JOIN nba_adv_box b ON a.nba_player_id = b.nba_player_id AND a.nba_game_id = b.nba_game_id
LEFT JOIN (SELECT DISTINCT(nba_game_id), nba_game_date, nba_season_id, nba_plus_minus, nba_gm_type FROM nba_team_box WHERE nba_matchup LIKE '% vs.%') c
ON a.nba_game_id = c.nba_game_id
LEFT JOIN (SELECT nba_person_id, nba_height, to_char(CAST(nba_birthdate as date), 'YYYY-MM-DD') birthdate FROM nba_common_attr) d
ON a.nba_player_id = d.nba_person_id
LEFT JOIN (SELECT br_slug, br_team, br_opponent, br_date, br_game_score, br_outcome, br_location FROM brid_tra_box) e
ON (a.nba_player_id = e.br_slug AND to_char(CAST(c.nba_game_date as date), 'YYYY-MM-DD') = to_char(CAST(e.br_date as date), 'YYYY-MM-DD'))
ORDER BY c.nba_game_date

"""

data = pd.read_sql(query, con=conn)
display(data.head(10))
data.info()

,nba_game_date,nba_season_id,br_location,nba_gm_type,br_outcome,home_plus_minus,nba_game_id,nba_team_id,nba_team_abbreviation,nba_team_city,...,nba_tm_tov_pct,nba_efg_pct,nba_ts_pct,nba_usg_pct,nba_e_pace,nba_pace,nba_pace_per40,nba_poss,nba_pie,br_game_score
0,1999-11-02,21999,AWAY,Regular,W,-20,29900013,1610612757,POR,Portland,...,7.7,0.875,0.875,0.114,88.39,87.66,73.05,41.0,0.204,10.8
1,1999-11-02,21999,AWAY,Regular,W,-7,29900002,1610612754,IND,Indiana,...,0.0,0.286,0.342,0.225,103.06,103.57,86.31,36.0,-0.053,0.9
2,1999-11-02,21999,AWAY,Regular,W,-7,29900011,1610612747,LAL,Los Angeles,...,14.3,0.250,0.307,0.240,92.61,89.30,74.42,21.0,-0.017,1.2
3,1999-11-02,21999,HOME,Regular,L,-13,29900006,1610612761,TOR,Toronto,...,0.0,0.438,0.438,0.174,92.14,95.14,79.28,40.0,0.174,5.2
4,1999-11-02,21999,HOME,Regular,L,-13,29900006,1610612761,TOR,Toronto,...,14.3,0.143,0.415,0.197,94.95,96.55,80.46,55.0,0.005,2.6
5,1999-11-02,21999,HOME,Regular,W,14,29900005,1610612766,CHH,Charlotte,...,10.0,0.409,0.478,0.257,104.18,103.43,86.19,90.0,0.153,15.8
6,1999-11-02,21999,HOME,Regular,L,-7,29900011,1610612762,UTA,Utah,...,20.0,0.333,0.333,0.133,84.84,86.64,72.20,25.0,0.000,1.6
7,1999-11-02,21999,HOME,Regular,W,6,29900004,1610612748,MIA,Miami,...,10.0,0.357,0.451,0.188,100.90,104.30,86.91,82.0,0.124,12.4
8,1999-11-02,21999,AWAY,Regular,W,-12,29900012,1610612760,SEA,Seattle,...,7.1,0.250,0.250,0.138,98.79,98.92,82.44,75.0,0.067,4.6
9,1999-11-02,21999,None,Regular,None,-7,29900002,1610612751,NJN,New Jersey,...,NaN,NaN,NaN,0.000,NaN,NaN,NaN,0.0,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 662482 entries, 0 to 662481
Data columns (total 58 columns):
nba_game_date            662482 non-null object
nba_season_id            662482 non-null object
br_location              553928 non-null object
nba_gm_type              662482 non-null object
br_outcome               553928 non-null object
home_plus_minus          662482 non-null int64
nba_game_id              662482 non-null object
nba_team_id              662482 non-null object
nba_team_abbreviation    662482 non-null object
nba_team_city            662482 non-null object
nba_player_id            662482 non-null object
nba_player_name          662482 non-null object
nba_start_position       269542 non-null object
nba_comment              107710 non-null object
nba_min                  662482 non-null int64
nba_fgm                  554771 non-null float64
nba_fga                  554771 non-null float64
nba_fg_pct               554771 non-null float64
nba_fg3m                

In [5]:
# data.to_csv('unprocessed.csv')